In [1]:
import pandas as pd
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
data = pd.read_csv('datasets/amazon/new_sampled.csv')
data.head()

,title,brand,asin
0,Chocolates Blueberries in Premium White Milk ...,Dilettante,B0016LFGHO
1,"Royal Lace 25302 Poster Board, 8.5 by 11-Inch...",Royal Lace,B003H2GFZ2
2,"Khazana Basmati Rice, 160-Ounce",Khazana,B004YUF8FE
3,"Dial 1071428 Zout Liquid Stain Remover, 16oz B...",Dial,B007IX9VRA
4,Norco Racks Components C-24U,Norco,B007US0X48


In [ ]:
# Building sentences
# 0 Chocolates Blueberries in Premium 
# 1 Royal Lace 25302
# [[Chocolates,Blueberries,in,Premium],
# [Royal,Lace,25302]]
titles = data['title'].values.tolist()
f = lambda x : x.split(' ')
sequences = list(map(f,titles))

[['',
  'Chocolates',
  'Blueberries',
  'in',
  'Premium',
  'White',
  'Milk',
  '&amp;',
  'Dark',
  'Dilettante',
  'Chocolate'],
 ['Royal',
  'Lace',
  '',
  '25302',
  'Poster',
  'Board,',
  '8.5',
  'by',
  '11-Inch,',
  'White,',
  '5-Pack'],
 ['Khazana', 'Basmati', 'Rice,', '160-Ounce'],
 ['Dial',
  '1071428',
  'Zout',
  'Liquid',
  'Stain',
  'Remover,',
  '16oz',
  'Bottle',
  '(Pack',
  'of',
  '12)'],
 ['', 'Norco', 'Racks', 'Components', 'C-24U'],
 ['Hamiltonbobs',
  'Premium',
  'Quality',
  '12V',
  'Pertronix',
  'Electronic',
  'Ignition',
  'IH',
  'International...'],
 ['5',
  'Star',
  'Clipboard',
  'Plastic',
  'Durable',
  'with',
  'Rounded',
  'Corners',
  'A4',
  'Blue'],
 ['PowerSmart',
  '2200mAh',
  'Battery',
  'Pack',
  'for',
  'SYMBOL',
  'MC9060G',
  'MC9090G',
  'MC9190G,',
  '21-65587-08,HBM-SYM9000L,'],
 ['', 'White', 'Truffle', 'Fabrique', 'Delices', 'Butter', '(3', 'ounce)'],
 ['',
  'Complete',
  'Starter',
  'Kit',
  '-',
  'Pear',
  'Oregon'

In [4]:
len(sequences)

98828

In [5]:
sequences = list(filter(lambda x: x != '',sequences))

In [10]:
model = gensim.models.Word2Vec(sequences, min_count=2,vector_size=200,epochs=10)

2024-11-09 20:18:02,644 : INFO : collecting all words and their counts
2024-11-09 20:18:02,645 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-11-09 20:18:02,652 : INFO : PROGRESS: at sentence #10000, processed 90541 words, keeping 26604 word types
2024-11-09 20:18:02,670 : INFO : PROGRESS: at sentence #20000, processed 181395 words, keeping 42453 word types
2024-11-09 20:18:02,702 : INFO : PROGRESS: at sentence #30000, processed 271550 words, keeping 55493 word types
2024-11-09 20:18:02,720 : INFO : PROGRESS: at sentence #40000, processed 361934 words, keeping 66867 word types
2024-11-09 20:18:02,739 : INFO : PROGRESS: at sentence #50000, processed 452396 words, keeping 77300 word types
2024-11-09 20:18:02,769 : INFO : PROGRESS: at sentence #60000, processed 542759 words, keeping 86764 word types
2024-11-09 20:18:02,792 : INFO : PROGRESS: at sentence #70000, processed 633201 words, keeping 95985 word types
2024-11-09 20:18:02,805 : INFO : PROGRESS: at s

In [11]:
model.wv.most_similar('Dell')

[('Canon', 0.9794471859931946),
 ('cartridge', 0.9791475534439087),
 ('MPI', 0.9713319540023804),
 ('Brother', 0.9689609408378601),
 ('Xerox', 0.9675341844558716),
 ('Mita', 0.9630378484725952),
 ('replacement', 0.9610087871551514),
 ('Printhead', 0.9603882431983948),
 ('Rhinotek', 0.9600849151611328),
 ('Kyocera', 0.9595034122467041)]

In [13]:
model.wv.similarity('HP','Dell')

0.9437876

In [14]:
model.wv.similarity('Xerox','Intel')

0.8406334

In [15]:
v = model.wv['HP'] - model.wv['Computer'] + model.wv['Laptop']

In [16]:
model.wv.most_similar(v)

[('HP', 0.9689770936965942),
 ('Dell', 0.9431419968605042),
 ('Canon', 0.9360847473144531),
 ('Brother', 0.9282299876213074),
 ('cartridge', 0.9277176856994629),
 ('(HP', 0.9133819937705994),
 ('Lexmark', 0.9086320996284485),
 ('Toner', 0.9081204533576965),
 ('samsung', 0.9064165353775024),
 ('Cartridge', 0.9051875472068787)]

In [18]:
model.wv.save_word2vec_format('models/embeddings/w2v.txt', binary=False)

2024-11-09 20:21:47,163 : INFO : storing 42467x200 projection weights into models/embeddings/w2v.txt


In [19]:
!python -m spacy init vectors en models/embeddings/w2v.txt models/spacy_embeddings --name title_vectors

ℹ Creating blank nlp object for language 'en'
✔ Successfully converted 42467 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
C:\Users\Yash Phatak\Desktop\EDAR\NER\brand-ner-main\models\spacy_embeddings


In [20]:
import spacy
nlp = spacy.load('models/spacy_embeddings')

In [21]:
nlp.add_pipe('ner')
nlp.to_disk('models/spacy_embeddings')

In [22]:
!python -m spacy train models/spacy_embeddings/config.cfg --output models/ner_amazon_embedding --paths.train datasets/amazon/spacy/train_shuffle.spacy --paths.dev datasets/amazon/spacy/dev_shuffle.spacy

✔ Created output directory: models\ner_amazon_embedding
ℹ Saving to output directory: models\ner_amazon_embedding
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['ner']
ℹ Initial learn rate: 0.001
E    #       LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  --------  ------  ------  ------  ------
  0       0     45.00    0.00    0.00    0.00    0.00
  0     200   2933.81   40.13   50.78   33.18    0.40
  0     400   2527.90   52.43   64.76   44.04    0.52
  0     600   2752.40   56.07   70.33   46.62    0.56
  0     800   3265.15   60.00   69.44   52.81    0.60
  0    1000   3615.55   60.85   74.15   51.59    0.61
  0    1200   4367.64   65.17   74.10   58.17    0.65
  0    1400   4969.50   68.21   71.43   65.26    0.68
  0    1600   6003.79   69.77   73.31   66.55    0.70
  0    1800   6926.50   70.69   75.16   66.72    0.